# Part d: regression on 1Dim ising modellusing a multilayer perceptron (neural network)

In [1]:
import numpy as np
import scipy.sparse as sp
np.random.seed(12)

import warnings
#Comment this to turn on warnings
warnings.filterwarnings('ignore')

from model_comparison import model_comparison
from resample import resample
import algorithms
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn


from ann import NeuralNetMLP
from utils import train_test_split
%matplotlib inline

In [2]:
### define Ising model aprams
# system size
L=40

# create 10000 random Ising states
states=np.random.choice([-1, 1], size=(10000,L))

def ising_energies(states,L):
    """
    This function calculates the energies of the states in the nn Ising Hamiltonian
    """
    J=np.zeros((L,L),)
    for i in range(L):
        J[i,(i+1)%L]-=1.0
    # compute energies
    E = np.einsum('...i,ij,...j->...',states,J,states)

    return E
# calculate Ising energies
energies=ising_energies(states,L)

"""
Comparing to project 1 - x = states, z = energies... 
Currently no y since we are in one dimension. 

"""

'\nComparing to project 1 - x = states, z = energies... \nCurrently no y since we are in one dimension. \n\n'

In [3]:
# reshape Ising states into RL samples: S_iS_j --> X_p
states=np.einsum('...i,...j->...ij', states, states)
shape=states.shape
states=states.reshape((shape[0],shape[1]*shape[2])) 
# build final data set
Data=[states,energies]

# define number of samples
n_samples=600
# define train and test data sets


In [4]:
X=Data[0]
Y=Data[1]

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, split_size=0.2)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((8000, 1600), (8000,), (2000, 1600), (2000,))

# Neural network 

In [6]:
# Experimental setup
metric = []

# eta > 0.001 causes divergense. 

lmdas = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]


ann = NeuralNetMLP(l2 = 0.01, 
                   batch_size=10, 
                   epochs=50, 
                   n_hidden=30,
                   #h_layers=[30,30,30],
                   eta = 0.0001)

ann.fit(X_train, y_train, X_valid, y_valid)
ann.predict(X_valid)
# returns a list of the mean mse score for different epochs or batches
metric.append(np.mean(ann.eval_["valid_preform"]))
print(np.mean(ann.eval_["valid_preform"]))
# returns mse for each batch


w1shape: (30, 1600)
w2shape: (30, 30)
w3shape: (30, 30)


ValueError: shapes (10,1) and (30,30) not aligned: 1 (dim 1) != 30 (dim 0)

# Comparing to scikitlearn MLPregression

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
mlp = MLPRegressor(max_iter=100)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_valid)

y_pred